In [1]:
#Importing libraries
import pandas as pd
import numpy as np

In [2]:
#Data Loading
df = pd.read_csv("visitas_tipificadas.csv")

In [3]:
df.head()

,id_camp,TripType,id_ticket,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,324078,3,106,Friday,6.811316e+10,1,FINANCIAL SERVICES,275.0
1,324079,3,106,Friday,6.053890e+10,1,FINANCIAL SERVICES,276.0
2,324080,3,121,Friday,6.811316e+10,1,FINANCIAL SERVICES,278.0
3,324081,3,121,Friday,6.811316e+10,1,FINANCIAL SERVICES,277.0
4,324082,3,153,Friday,6.811311e+10,1,FINANCIAL SERVICES,200.0


In [4]:
df = df[['id_camp','id_ticket','Weekday','Upc','ScanCount','DepartmentDescription','FinelineNumber','TripType']]
df.head()

,id_camp,id_ticket,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,TripType
0,324078,106,Friday,6.811316e+10,1,FINANCIAL SERVICES,275.0,3
1,324079,106,Friday,6.053890e+10,1,FINANCIAL SERVICES,276.0,3
2,324080,121,Friday,6.811316e+10,1,FINANCIAL SERVICES,278.0,3
3,324081,121,Friday,6.811316e+10,1,FINANCIAL SERVICES,277.0,3
4,324082,153,Friday,6.811311e+10,1,FINANCIAL SERVICES,200.0,3


In [5]:
#Dimensions
print('Shape:', df.shape)
df.shape

Shape: (646654, 8)


(646654, 8)

In [6]:
#Null Values
print('Null Values')
df.isnull().sum()

Null Values


id_camp                     0
id_ticket                   0
Weekday                     0
Upc                      4129
ScanCount                   0
DepartmentDescription    1361
FinelineNumber           4129
TripType                    0
dtype: int64

In [7]:
#Variable types
print('Variable Types')
df.dtypes

Variable Types


id_camp                    int64
id_ticket                  int64
Weekday                   object
Upc                      float64
ScanCount                  int64
DepartmentDescription     object
FinelineNumber           float64
TripType                   int64
dtype: object

In [8]:
missing = df.isna().sum()
missing = missing[missing>0]
missing_perc = missing/df.shape[0]*100
na = pd.DataFrame([missing, missing_perc], index = ['missing_num', 'missing_perc']).T
na = na.sort_values(by = 'missing_perc', ascending = False)
print(na)

                       missing_num  missing_perc
Upc                         4129.0      0.638518
FinelineNumber              4129.0      0.638518
DepartmentDescription       1361.0      0.210468


In [9]:
# Debido a que los N/A representan un porcentaje mínimo de la columna, pueden eliminarse.
df.dropna(inplace=True)

In [10]:
df.TripType.unique()

array([  3,   4,   5,   6,   7,   8,   9,  12,  14,  15,  18,  19,  20,
        21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,
        34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44, 999])

In [11]:
df.TripType.value_counts()

40     173857
39      95324
37      38864
38      29505
25      27482
7       23158
8       22751
36      21922
44      20351
42      19410
24      17952
999     17226
9       16730
32      13802
35      12486
5       11240
33       9897
15       7118
3        6815
43       6340
41       5486
30       4840
34       4733
27       4599
21       4023
22       3578
6        3398
20       3107
18       2968
28       2658
26       2502
12       2103
29       2098
31       1760
19       1187
4         895
23        325
14         35
Name: TripType, dtype: int64

In [12]:
df.TripType.astype('O')

0           3
1           3
2           3
3           3
4           3
         ... 
646649    999
646650    999
646651    999
646652    999
646653    999
Name: TripType, Length: 642525, dtype: object

In [13]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df.Weekday.unique())
df.Weekday=le.transform(df.Weekday)

In [14]:
df.Weekday.unique()

array([0, 2, 3, 1, 5, 6, 4])

In [15]:
le2 = preprocessing.LabelEncoder()
le2.fit(df.DepartmentDescription.unique())
df.DepartmentDescription = le2.transform(df.DepartmentDescription)

In [16]:
df.DepartmentDescription.unique()

array([20, 32, 38, 30,  3, 50, 67, 17, 21, 15,  2,  4, 11, 57, 40, 43, 24,
       64, 12, 31, 25, 60, 18, 52, 46, 63,  7, 13, 62,  5, 51, 39,  8, 23,
       35, 33, 16, 28, 45, 19, 41, 27, 14, 10,  6, 34, 42,  0, 55, 66, 49,
       37,  1, 54, 44, 22, 65, 29, 56, 53, 47, 36,  9, 58, 61, 59, 48, 26])

In [17]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

xval = df.values[:,0:-1]
yval = df.values[:,-1]
x = np.asanyarray(xval)
y = np.asanyarray(yval)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=0)

In [18]:
from sklearn import tree
dt = tree.DecisionTreeClassifier(random_state=0)
dt.fit(X_train, y_train)
dt_pred= dt.predict(X_test)
print(accuracy_score(y_test,dt_pred)*100)


99.99151079543847


In [20]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
clf_pred=clf.predict(X_test)
print(accuracy_score(y_test,clf_pred)*100) 

43.45057868077761


In [26]:
#F1, RECALL Y PRECISION (Las cuales son más usadas para medir el desempeño cuando la base de datos está desbalanceada)
from sklearn.metrics import classification_report
print(classification_report(y_test, dt_pred))

              precision    recall  f1-score   support

         3.0       1.00      1.00      1.00      2242
         4.0       1.00      0.99      1.00       269
         5.0       1.00      1.00      1.00      3733
         6.0       1.00      1.00      1.00      1117
         7.0       1.00      1.00      1.00      7695
         8.0       1.00      1.00      1.00      7374
         9.0       1.00      1.00      1.00      5529
        12.0       1.00      1.00      1.00       687
        14.0       1.00      1.00      1.00        10
        15.0       1.00      1.00      1.00      2334
        18.0       1.00      1.00      1.00      1021
        19.0       1.00      1.00      1.00       368
        20.0       1.00      1.00      1.00      1038
        21.0       1.00      1.00      1.00      1316
        22.0       1.00      1.00      1.00      1223
        23.0       0.99      1.00      1.00       112
        24.0       1.00      1.00      1.00      5968
        25.0       1.00    

In [27]:
print(classification_report(y_test, clf_pred))

/Users/oscaraguilar/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         3.0       0.00      0.00      0.00      2242
         4.0       0.00      0.00      0.00       269
         5.0       0.00      0.00      0.00      3733
         6.0       0.00      0.00      0.00      1117
         7.0       0.00      0.00      0.00      7695
         8.0       0.00      0.00      0.00      7374
         9.0       0.00      0.00      0.00      5529
        12.0       0.00      0.00      0.00       687
        14.0       0.00      0.00      0.00        10
        15.0       0.00      0.00      0.00      2334
        18.0       0.00      0.00      0.00      1021
        19.0       0.00      0.00      0.00       368
        20.0       0.00      0.00      0.00      1038
        21.0       0.00      0.00      0.00      1316
        22.0       0.00      0.00      0.00      1223
        23.0       0.00      0.00      0.00       112
        24.0       0.00      0.00      0.00      5968
        25.0       0.00    

La razón por la cual los modelos tienen un desempeño tan diferente es porque
1. Hay un número considerable de clases
2. Hay clases con mucho mayor proporción de instancias que otras i.e. la clase "40" con 173857 casos y la clase 14 con 35 (class imbalance/ desbalance en clases)
3. Al ver 99.99% de exactitud, sospeché que el árbol de decisión estaba prediciendo únicamente la clase más común en la base de datos, por lo que el resultado del modelo de ensamble (Random forest) es más real.

El siguiente paso es trabajar el desbalance de clase con upsampling o downsampling

In [ ]:
from sklearn.utils import resample
df_imbalanced=df[df.TripType != '14', '23', '4', '40', '39', '37']

df_14 = df[df.TripType == '14']
df_14_upsampled = resample(df_14, 
                           replace=True,     # sample with replacement
                           n_samples=5000,    # to match majority class
                           random_state=0) # reproducible results
df_23 = df[df.TripType == '23']
df_23_upsampled = resample(df_23, 
                           replace=True,     # sample with replacement
                           n_samples=5000,    # to match majority class
                           random_state=0) # reproducible results
df_4 = df[df.TripType == '4']
df_4_upsampled = resample(df_4, 
                           replace=True,     # sample with replacement
                           n_samples=5000,    # to match majority class
                           random_state=0) # reproducible results

df_40 = df[df.TripType == '40']
df_40_downsampled = resample(df_40, 
                             replace=False,    # sample without replacement
                             n_samples=30000,     # to match minority class
                             random_state=0) # reproducible results

df_39 = df[df.TripType == '39']
df_39_downsampled = resample(df_39, 
                             replace=False,    # sample without replacement
                             n_samples=30000,     # to match minority class
                             random_state=0) # reproducible results

df_37 = df[df.TripType == '37']
df_37_downsampled = resample(df_37, 
                             replace=False,    # sample without replacement
                             n_samples=30000,     # to match minority class
                             random_state=0) # reproducible results
 

# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df, df_14_upsampled, df_23_upsampled, df_4_upsampled])
df_final = pd.concat([df_imbalanced, df_upsampled, df_40_downsampled, df_39_downsampled, df_37_downsampled])
 
# Display new class counts
df_final.TripType.value_counts()

# No alcancé a lidiar con el desbalance de clases, pero esto hubiera seguido. Reentrenar.

In [34]:
xval2 = df_final.values[:,0:-1]
yval2 = df_final.values[:,-1]
x2 = np.asanyarray(xval2)
y2 = np.asanyarray(yval2)
X_train2, X_test2, y_train2, y_test2 = train_test_split(x2, y2, test_size=0.33, random_state=0)

In [ ]:
clf2 = RandomForestClassifier(max_depth=2, random_state=0)
clf2.fit(X_train2, y_train2)
clf_pred2=clf2.predict(X_test2)
print(accuracy_score(y_test2,clf_pred2)*100) 

# Llenaré el excel con el modelo de random forest

In [44]:
#Read the data
newtest = pd.read_csv("visitas_agosto.csv", sep=',',header= None)
#Fit the data
newtest=newtest.iloc[1:]
xnew=newtest.values[:,-1]
xnew=np.array(xnew, dtype=float)
#Run the SVM (Most accurate classifier)
results=clf.predict(xnew)
#Export results to a .csv file
df=pd.DataFrame(results)
df.to_csv("visitas_tipic_ago.csv")

ValueError: Expected 2D array, got 1D array instead:
array=[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan 